# Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from requests_html import HTMLSession

import re
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from time import sleep
import random
import pickle

import psycopg2 as pg2

pd.set_option('display.max_columns', 100)

## First we need the names of athletes that fought in ufc, we can get them from ufc's site

In [ ]:
'''
The names which we are intersted are loading dynamically with a button.
We can use selenium to load them all and then scrape them.

'''


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # Open chrome

driver.get("https://www.ufc.com/athletes/all") # Go to the webpage

driver.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click() # Deny cookies


while True:
    
    try:
        # Find and click the "Load More" button
        driver.find_element(By.XPATH,
                            '/html/body/div[1]/div/main/div[1]/div/div/div/div/div/div/div/div/div[2]/div/div/ul/li/a').click()
        
        sleep(10 + random.random()) 
    except:
        # If the button is not found or an error occurs, break the loop
        break

In [ ]:
'''
Now we can use BeautifulSoup to scrape them

'''

# Parse the updated page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# How many names there are
objects = len(soup.findAll('span', {'class' : "c-listing-athlete__name"}))

# We use step=2 to speed up the process since in the span class we look there are more than once the athletes names 
athlete_objects = ([soup.findAll('span', {'class' : "c-listing-athlete__name"})[i].text.strip()
                    for i in tqdm(range(0, objects, 2))])

# Unique list of athletes names
athlete_names = list(set(athlete_objects))

In [ ]:
with open("D:/data_projects/mma/athletes_names_list.pickle", "wb") as file:
    pickle.dump(athlete_names, file)

In [2]:
with open("D:/data_projects/mma/athletes_names_list.pickle", "rb") as file:
    athletes_list = pickle.load(file)

In [3]:
print('We have', len(athletes_list), 'athletes')

We have 2773 athletes


## We can also make their urls from their names for the same site to use later

In [ ]:
url_athletes_list = [re.sub(r'[^\w\s]', '', name.lower()) for name in athletes_list] # Remove non words and spaces
url_athletes_list = [name.strip() for name in url_athletes_list] # Remove spaces left and right
url_athletes_list = ['https://www.ufc.com/athlete/' + '-'.join(name.split()) for name in url_athletes_list] # Make urls

In [ ]:
url_athletes_list

In [ ]:
with open("D:/data_projects/mma/athletes_url.pickle", "wb") as file:
    pickle.dump(url_athletes_list, file)

In [ ]:
with open("D:/data_projects/mma/athletes_url.pickle", "rb") as file:
    url_athletes = pickle.load(file)

## Now we want to get their stats. We have 3 sites we will use : ufc, ufcstats, espn

## Get all fighters links from espn

In [19]:
'''
Espn have them categorized by the alphabet letters which makes it a lot easier

'''

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
fighters_links_list = []

for letter in tqdm(letters):
    
    main_link = 'http://www.espn.com/mma/fighters?search=' + letter
    headers = {'User-agent': ''}
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # For each letter of the alphabet check the links which appear and append in the list those which belong to fighters
    fighters_links_list += ['https://www.espn.com' + link.get("href") for link in soup.find_all("a") if '/mma/fighter/_/id/' in link.get("href")]
    
    sleep(5 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [03:07<00:00,  7.21s/it]


In [20]:
len(fighters_links_list)

31501

### We can see we have more than 10 times the athletes we got from ufc site as espn has more organizations

## We'll deal with this later, lets build our function fow now

In [2]:
def get_fighters_df(url):
    
    '''
    Inputs a url from espn site of an mma athlete and returns a dataframe with statistics
    
    '''
    
    main_link = url.split('fighter')[0] + 'fighter/stats' + url.split('fighter')[1] # Starting point for each athlete
    headers = {'User-agent': ''}
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    sleep(3 + random.random())

    # If this table exists get all those below
    if soup.find('div', class_='ResponsiveTable pt4'):

        try: name = ' '.join([span.text for span in soup.find('h1', {'class': 'PlayerHeader__Name'}).find_all('span')])
        except: name = np.nan

        try: country = soup.find('li', class_='truncate min-w-0').text
        except: country = np.nan

        try: division = soup.find('li', class_='').text
        except: division = np.nan

        try: height = soup.find('div', class_='ttu', text='HT/WT').find_next_sibling('div').text.split(',')[0] # Height
        except: height = np.nan

        try: weight = soup.find('div', class_='ttu', text='HT/WT').find_next_sibling('div').text.split(',')[1] # Weight
        except: weight = np.nan

        try: birth_date = soup.find('div', class_='ttu', text='Birthdate').find_next_sibling('div').text # Birth date
        except: birth_date = np.nan

        try: team = soup.find('div', class_='ttu', text='Team').find_next_sibling('div').text # Team
        except: team = np.nan

        try: stance = soup.find('div', class_='ttu', text='Stance').find_next_sibling('div').text # Stance
        except: stance = np.nan

        try: reach = soup.find('div', class_='ttu', text='Reach').find_next_sibling('div').text # Reach
        except: reach = np.nan



        try: wins_loses_draws = soup.find('div', attrs={"aria-label": "Wins-Losses-Draws"}).find_next_sibling('div').text # W-L-D
        except: wins_loses_draws = np.nan

        try: ko_tko_wins = soup.find("div", attrs={"aria-label": "Technical Knockout-Technical Knockout Losses"}).find_next_sibling("div").text.split('-')[0] # (T)KO wins
        except: ko_tko_wins = np.nan

        try: ko_tko_loses = soup.find("div", attrs={"aria-label": "Technical Knockout-Technical Knockout Losses"}).find_next_sibling("div").text.split('-')[0] # (T)KO loses
        except: ko_tko_loses = np.nan

        try: submission_wins = soup.find("div", attrs={"aria-label": "Submissions-Submission Losses"}).find_next_sibling("div").text.split('-')[0] # Submission wins 
        except: submission_wins = np.nan

        try: submission_loses = soup.find("div", attrs={"aria-label": "Submissions-Submission Losses"}).find_next_sibling("div").text.split('-')[1] # Submission loses
        except:  submission_loses = np.nan




        try:
            # Find the striking table if it exists
            striking_table = soup.select_one(".ResponsiveTable:-soup-contains('striking') table")

            # If its not empty
            if striking_table:
                # Extract the headers of the table
                headers_striking = [th.text.strip() for th in striking_table.select("thead th")]

                # Extract the rows of the table
                rows_striking = []
                for tr in striking_table.select("tbody tr"):
                    cells_striking = [td.text.strip() for td in tr.select("td")]
                    rows_striking.append(cells_striking)
        except:
            pass

        try:
            # Find the clinch table if it exists
            clinch_table = soup.select_one(".ResponsiveTable:-soup-contains('Clinch') table")

            # If its not empty
            if clinch_table:
                # Extract the headers of the table
                headers_clinch = [th.text.strip() for th in clinch_table.select("thead th")]

                # Extract the rows of the table
                rows_clinch = []
                for tr in clinch_table.select("tbody tr"):
                    cells_clinch = [td.text.strip() for td in tr.select("td")]
                    rows_clinch.append(cells_clinch)
        except:
            pass

        try:
            # Find the ground table if it exists
            ground_table = soup.select_one(".ResponsiveTable:-soup-contains('Ground') table")

            # If its not empty
            if ground_table:
                # Extract the headers of the table
                headers_ground = [th.text.strip() for th in ground_table.select("thead th")]

                # Extract the rows of the table
                rows_ground = []
                for tr in ground_table.select("tbody tr"):
                    cells_ground = [td.text.strip() for td in tr.select("td")]
                    rows_ground.append(cells_ground)
        except:
            pass

        # Go to fight history page
        main_link = url.split('fighter')[0] + 'fighter/history' + url.split('fighter')[1]
        headers = {'User-agent': ''}
        response = requests.get(main_link, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        sleep(3 + random.random())

        try:
            # Find the fight history table if it exists
            fight_history_table = soup.select_one(".ResponsiveTable:-soup-contains('Fight') table")

            # If its not empty
            if fight_history_table:
                # Extract the headers of the table
                headers_fight_history = [th.text.strip() for th in fight_history_table.select("thead th")]

                # Extract the rows of the table
                rows_fight_history = []
                for tr in fight_history_table.select("tbody tr"):
                    cells_fight_history = [td.text.strip() for td in tr.select("td")]
                    rows_fight_history.append(cells_fight_history)
        except:
            pass    



        '''
        Finally we make 3 dataframes and then combine them to 1
        
        '''
        
        try:
            # Dataframe of the striking, clinch, ground tables with columns names the headers which we extracted
            temp_tables = pd.DataFrame(rows_striking, columns=headers_striking).merge(
                              pd.DataFrame(rows_clinch, columns=headers_clinch), on=['Date', 'Opponent', 'Event', 'Res.']).merge(
                                  pd.DataFrame(rows_ground, columns=headers_ground), on=['Date', 'Opponent', 'Event', 'Res.'])

            # Dataframe of name country etc.. (these values wont change for each fighter)
            temp_stats = pd.DataFrame([[name, country, division, height, weight, birth_date, stance, reach]] * len(temp_tables),
                              columns=['name', 'country', 'division', 'height', 'weight', 'birth_date', 'stance', 'reach'])

            # Dataframe of fight history
            temp_fight_history = pd.DataFrame(rows_fight_history, columns=headers_fight_history)
            # Exclude the EVENT column
            temp_fight_history = temp_fight_history.loc[:, temp_fight_history.columns != 'Event']

            # Combine 3 dataframes to 1
            df= temp_stats.merge(
                    temp_tables.merge(
                        temp_fight_history, on=['Date', 'Opponent', 'Res.']), left_index=True, right_index=True)
            return df

        except:
            pass   

## Do some tests

In [3]:
df1 = get_fighters_df('https://www.espn.com/mma/fighter/_/id/3332412/islam-makhachev')

In [4]:
df2 = get_fighters_df('https://www.espn.com/mma/fighter/_/id/2504229/nik-lentz')

In [5]:
df3 = get_fighters_df('https://www.espn.com/mma/fighter/_/id/4239497/tommy-aaron')

In [16]:
my_list = [
         'https://www.espn.com/mma/fighter/_/id/3332412/islam-makhachev',
         'https://www.espn.com/mma/fighter/_/id/2504229/nik-lentz',
         'https://www.espn.com/mma/fighter/_/id/4239497/tommy-aaron'
            ]

In [18]:
pd.concat(list(map(get_fighters_df, tqdm(my_list))))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:21<00:00,  7.09s/it]


,name,country,division,height,weight,birth_date,stance,reach,Date,Opponent,...,SGLA,AD,ADTB,ADHG,ADTM,ADTS,SM,Decision,Rnd,Time
0,Islam Makhachev,Russia,Lightweight,"5' 10""",155 lbs,10/27/1991 (31),Southpaw,"70.5""","Oct 22, 2022",Charles Oliveira,...,0,0,0,0,0,0,1,Submission,2,3:16
1,Islam Makhachev,Russia,Lightweight,"5' 10""",155 lbs,10/27/1991 (31),Southpaw,"70.5""","Feb 26, 2022",Bobby Green,...,0,0,0,0,0,0,0,KO/TKO,1,3:23
2,Islam Makhachev,Russia,Lightweight,"5' 10""",155 lbs,10/27/1991 (31),Southpaw,"70.5""","Oct 30, 2021",Dan Hooker,...,0,0,0,0,0,0,2,Submission,1,2:25
3,Islam Makhachev,Russia,Lightweight,"5' 10""",155 lbs,10/27/1991 (31),Southpaw,"70.5""","Jul 17, 2021",Thiago Moises,...,0,0,0,0,0,0,1,Submission,4,2:38
4,Islam Makhachev,Russia,Lightweight,"5' 10""",155 lbs,10/27/1991 (31),Southpaw,"70.5""","Mar 6, 2021",Drew Dober,...,0,0,0,0,0,0,2,Submission,3,1:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,Nik Lentz,USA,Featherweight,"5' 8""",150 lbs,8/13/1984 (38),Orthodox,NaN,"Feb 1, 2006",Kenneth Allen,...,-,-,-,-,-,-,-,Submission (Rear Naked Choke),2,-
41,Nik Lentz,USA,Featherweight,"5' 8""",150 lbs,8/13/1984 (38),Orthodox,NaN,"Dec 5, 2005",Derek Abram,...,-,-,-,-,-,-,-,TKO,1,-
42,Nik Lentz,USA,Featherweight,"5' 8""",150 lbs,8/13/1984 (38),Orthodox,NaN,"Oct 10, 2005",Nick Melton,...,-,-,-,-,-,-,-,TKO (Punches),1,-
43,Nik Lentz,USA,Featherweight,"5' 8""",150 lbs,8/13/1984 (38),Orthodox,NaN,"Aug 27, 2005",Anthony White,...,-,-,-,-,-,-,-,TKO,1,-


### Use the names scraped from ufc site to reduce the size of links, run only fighters which fought in the ufc

In [51]:
with open("D:/data_projects/mma/athletes_names_list.pickle", "rb") as file:
    athletes_list_fought_ufc = pickle.load(file)

In [69]:
fought_ufc_strings = ['-'.join(athlete.lower().split()) for athlete in athletes_list_fought_ufc]

## The fighters urls we are interested in

In [73]:
# Keep only the links which are for fighters who fought in the ufc
target_fighters_urls = [link for fighter in fought_ufc_strings for link in fighters_links_list if fighter in link]

In [74]:
with open("D:/data_projects/mma/fighters_links_list.pickle", "wb") as file:
    pickle.dump(target_fighters_urls, file)

In [75]:
with open("D:/data_projects/mma/fighters_links_list.pickle", "rb") as file:
    target_fighters_urls = pickle.load(file)

## Create the dataframe

In [76]:
df = pd.concat(list(map(get_fighters_df, tqdm(target_fighters_urls))))

100%|████████████████████████████████████████████████████████████████████████████| 2751/2751 [5:47:02<00:00,  7.57s/it]


In [77]:
with open("D:/data_projects/mma/fighters_dataframe.pickle", "wb") as file:
    pickle.dump(df, file)

In [188]:
with open("D:/data_projects/mma/fighters_dataframe.pickle", "rb") as file:
    df = pickle.load(file)

### We want to keep only UFC fights as there is much more information than other organizations
    
    So we dont keep data of fighters who fought in other organizations too beside the ufc

In [189]:
df = df[df['Event'].str.contains('UFC')]

In [190]:
df.head()

,name,country,division,height,weight,birth_date,stance,reach,Date,Opponent,Event,Res.,SDBL/A,SDHL/A,SDLL/A,TSL,TSA,SSL,SSA,TSL-TSA,KD,%BODY,%HEAD,%LEG,SCBL,SCBA,SCHL,SCHA,SCLL,SCLA,RV,SR,TDL,TDA,TDS,TK ACC,SGBL,SGBA,SGHL,SGHA,SGLL,SGLA,AD,ADTB,ADHG,ADTM,ADTS,SM,Decision,Rnd,Time,SDBL/A_x,SDHL/A_x,SDLL/A_x,TSL_x,TSA_x,SSL_x,SSA_x,TSL-TSA_x,KD_x,%BODY_x,%HEAD_x,%LEG_x,SDBL/A_y,SDHL/A_y,SDLL/A_y,TSL_y,TSA_y,SSL_y,SSA_y,TSL-TSA_y,KD_y,%BODY_y,%HEAD_y,%LEG_y
0,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Sep 10, 2022",Chris Barnett,UFC 279,L,5/7,22/62,1/1,57,107,51,98,53.27%,1,83%,47%,100%,5,5,8,11,0,0,0,0.000,1,2,0,50%,0,0,10,12,0,0,0,0,0,0,0,1,KO/TKO,2,2:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Apr 30, 2022",Andrei Arlovski,UFC Fight Night,L,4/7,68/155,9/9,132,231,93,188,57.14%,0,70%,46%,90%,3,3,9,13,0,1,0,0.000,2,3,0,67%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,5:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Jan 15, 2022",Chase Sherman,UFC Fight Night,W,2/2,10/33,0/1,20,50,20,49,40.00%,0,100%,39%,0%,0,0,0,0,0,0,0,0.000,1,1,0,100%,0,0,8,13,0,0,0,0,0,0,0,1,Submission,1,2:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Jun 12, 2021",Carlos Felipe,UFC 263,L,18/24,85/227,24/26,130,281,130,281,46.26%,0,77%,38%,92%,2,2,1,2,0,0,0,0.000,0,1,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,5:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Dec 5, 2020",Gian Villante,UFC Fight Night,W,18/28,87/224,16/18,126,277,123,272,45.49%,0,66%,39%,89%,1,1,1,1,0,0,0,0.000,0,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Unanimous,3,5:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### These fighters are making some problems to our dataframe generating some extra columns

In [191]:
df[df['%LEG_y'].notnull()]['name'].unique()

array(['Chris Gutierrez', 'Brock Larson', 'Mike Malott',
       'Luigi Fioravanti'], dtype=object)

#### Lets find a way to fix this

In [192]:
'''
In those extra column names there is the pattern '_x' and _y, those extra columns are created when we try to merge 2 dataframes,
which have same column names, and we dont specify to merge on those collumns too.

We also see that only for those 4 athletes there are non null values in those '_x' and '_y' collumns. Everything else is null.

There is probably some differences in the html format of those 4 athletes and its beyond of the scope of this project to find
out for sure what it is, so we assume that and we'll do a workaround to fix it.

'''

proper_columns = ['SDBL/A', 'SDHL/A', 'SDLL/A', 'TSL', 'TSA', 'SSL', 'SSA', 'TSL-TSA', 'KD', '%BODY', '%HEAD', '%LEG']
excessive_columns = [col+'_x' for col in proper_columns] # [col+'_y' for col in proper_columns]

In [193]:
def fixing_collumns(col, col_to_fix):
    
    '''
    Inputs a column and a column to fix ('_x', '_y' etc..).
    If a value of the column to fix is not null leave it as it is, else replace it with the value of the proper column
    Outputs the updated column (pandas Series)
    
    '''
    
    df[col] = np.where(df[col_to_fix].notnull(), df[col_to_fix], df[col])    
    
    return df[col]

## Apply this function to update the columns

In [194]:
for col_to_fix in excessive_columns:
    
    df[col_to_fix.split('_')[0]] = fixing_collumns(col_to_fix.split('_')[0], col_to_fix)

# Drop the columns we dont need anymore
cols_to_drop = [col+'_x' for col in proper_columns] + [col+'_y' for col in proper_columns]
df.drop(cols_to_drop, axis=1, inplace = True)

## Lets now check for null values

In [195]:
for k,v in df.isnull().sum().items():
    print(k,v)

name 0
country 1499
division 1
height 2841
weight 2841
birth_date 159
stance 41
reach 5201
Date 0
Opponent 0
Event 0
Res. 0
SDBL/A 0
SDHL/A 0
SDLL/A 0
TSL 0
TSA 0
SSL 0
SSA 0
TSL-TSA 0
KD 0
%BODY 0
%HEAD 0
%LEG 0
SCBL 0
SCBA 0
SCHL 0
SCHA 0
SCLL 0
SCLA 0
RV 0
SR 0
TDL 0
TDA 0
TDS 0
TK ACC 0
SGBL 22
SGBA 22
SGHL 22
SGHA 22
SGLL 22
SGLA 22
AD 22
ADTB 22
ADHG 22
ADTM 22
ADTS 22
SM 22
Decision 0
Rnd 0
Time 0


## There are many nulls with
    country
    division
    height
    weight
    birth_date
    stance
    reach
    
## We have to scrape more to get those information

## Scrape from ufc site

In [2]:
with open("D:/data_projects/mma/athletes_url.pickle", "rb") as file:
    ufc_url_athletes = pickle.load(file)

In [3]:
def get_stats_ufc(link):
    
    '''
    Inputs a url for a fighter in the ufc page.
    Outputs a dataframe with statistics.
    
    '''
    
    main_link = link
    headers = {'User-agent': ''}
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    sleep(3 + random.random())
    
    
    try: name = ' '.join(main_link.split('/')[-1].split('-')).title()
    except: name = np.nan

    try: hometown = soup.find('div', class_='c-bio__label', text='Hometown').find_next_sibling('div', class_='c-bio__text').text # Hometown
    except: hometown = np.nan

    try: fighting_style = soup.find('div', class_='c-bio__label', text='Fighting style').find_next_sibling('div', class_='c-bio__text').text # Style
    except: fighting_style = np.nan

    try: height = soup.find('div', class_='c-bio__label', text='Height').find_next_sibling('div', class_='c-bio__text').text # Height
    except: height = np.nan

    try: weight = soup.find('div', class_='c-bio__label', text='Weight').find_next_sibling('div', class_='c-bio__text').text # Weight
    except: weight = np.nan

    try: reach = soup.find('div', class_='c-bio__label', text='Reach').find_next_sibling('div', class_='c-bio__text').text # Reach
    except: reach = np.nan

    try: leg_reach = soup.find('div', class_='c-bio__label', text='Leg reach').find_next_sibling('div', class_='c-bio__text').text # Reach
    except: leg_reach = np.nan
        
        
    df = pd.DataFrame([[name, hometown, fighting_style, height, weight, reach, leg_reach]],
                              columns=['name', 'hometown', 'fighting_style', 'height', 'weight', 'reach', 'leg_reach'])
    
    return df

In [4]:
ufc_athletes_stats_df = pd.concat(list(map(get_stats_ufc, tqdm(ufc_url_athletes))), ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████| 2773/2773 [4:08:28<00:00,  5.38s/it]


In [6]:
with open("D:/data_projects/mma/ufc_athletes_stats_df.pickle", "wb") as file:
    pickle.dump(ufc_athletes_stats_df, file)

In [196]:
with open("D:/data_projects/mma/ufc_athletes_stats_df.pickle", "rb") as file:
    ufc_athletes_stats_df = pickle.load(file)

In [197]:
ufc_athletes_stats_df.tail()

,name,hometown,fighting_style,height,weight,reach,leg_reach
2768,Edson Gomez,NaN,NaN,NaN,174.00,NaN,NaN
2769,Kiichi Kunimoto,Japan,NaN,69.00,170.00,71.00,41.00
2770,Cole Williams,"Elkhorn, United States",NaN,72.00,175.50,73.00,39.00
2771,Isaac Vallieflagg,NaN,NaN,NaN,NaN,NaN,NaN
2772,Beneil Dariush,United States,MMA,70.00,156.00,72.00,40.50


## Scrape from ufc stats site

In [222]:
'''
Get the fighters links

'''

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ufcstats_list = []

for letter in tqdm(letters):
    
    main_link = 'http://ufcstats.com/statistics/fighters?char=' + letter + '&page=all'
    headers = {'User-agent': ''}
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    ufcstats_list += [link.get('href') for link in soup.find_all('a') if link.get('href') and '/fighter-details/' in link.get('href')]
    
    sleep(3 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:52<00:00,  4.32s/it]


In [223]:
ufcstats_list = list(set(ufcstats_list))

In [224]:
with open("D:/data_projects/mma/ufcstats_list.pickle", "wb") as file:
    pickle.dump(ufcstats_list, file)

In [3]:
with open("D:/data_projects/mma/ufcstats_list.pickle", "rb") as file:
    ufcstats_list = pickle.load(file)

### Make the function

In [4]:
def get_stats_ufcstats(link):
    
    '''
    Inputs a url for a fighter in the ufcstats page.
    Outputs a dataframe with statistics.
    
    '''
        
    main_link = link
    headers = {'User-agent': ''}
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    sleep(3 + random.random()) # they are too many to give a bigger sleep

    try: name = soup.find('span', class_='b-content__title-highlight').text.strip() # Name
    except: name = np.nan

    try: height = soup.select_one("li:-soup-contains('Height')").text.replace("Height:", "").strip()
    except: height = np.nan

    try: weight = soup.select_one("li:-soup-contains('Weight')").text.replace("Weight:", "").strip()
    except: weight = np.nan

    try: reach = soup.select_one("li:-soup-contains('Reach')").text.replace("Reach:", "").strip()
    except: reach = np.nan

    try: stance = soup.select_one("li:-soup-contains('STANCE')").text.replace("STANCE:", "").strip()
    except: stance = np.nan

    try: date_of_birth = soup.select_one("li:-soup-contains('DOB')").text.replace("DOB:", "").strip()
    except: date_of_birth = np.nan
        
    df = pd.DataFrame([[name, height, weight, reach, stance, date_of_birth]],
                              columns=['name', 'height', 'weight', 'reach', 'stance', 'date_of_birth'])
    
    return df

In [5]:
ufcstats_df = pd.concat(list(map(get_stats_ufcstats, tqdm(ufcstats_list))), ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████| 3922/3922 [4:26:10<00:00,  4.07s/it]


In [6]:
with open("D:/data_projects/mma/ufcstats_df.pickle", "wb") as file:
    pickle.dump(ufcstats_df, file)

In [198]:
with open("D:/data_projects/mma/ufcstats_df.pickle", "rb") as file:
    ufcstats_df = pickle.load(file)

In [199]:
ufcstats_df.tail()

,name,height,weight,reach,stance,date_of_birth
3917,Johnny Case,"5' 10""",155 lbs.,"72""",Orthodox,"Jun 28, 1989"
3918,Sultan Aliev,"5' 11""",170 lbs.,"74""",Orthodox,"Sep 17, 1984"
3919,Theo Rlayang,"5' 7""",145 lbs.,"65""",Orthodox,"Aug 10, 1994"
3920,Rolles Gracie,"6' 4""",242 lbs.,--,Orthodox,"Jul 14, 1978"
3921,Katsuyori Shibata,"6' 0""",168 lbs.,--,Orthodox,"Nov 17, 1979"


# Lets now combine our dataframes

In [200]:
# These are the names of the athletes which we are interested

names_to_use = df['name'].unique()

In [201]:
# Keep only those names in the other two dataframes

ufc_athletes_stats_df = ufc_athletes_stats_df[ufc_athletes_stats_df['name'].isin(names_to_use)]

ufcstats_df = ufcstats_df[ufcstats_df['name'].isin(names_to_use)]

In [202]:
# We do outer merge to keep all information and combine the same columns

df = pd.merge(
        df, pd.merge(ufc_athletes_stats_df, ufcstats_df, how='outer', on='name'),
            how='outer', on='name'
        )

# Data Cleaning

In [204]:
df.head()

,name,country,division,height,weight,birth_date,stance_x,reach,Date,Opponent,Event,Res.,SDBL/A,SDHL/A,SDLL/A,TSL,TSA,SSL,SSA,TSL-TSA,KD,%BODY,%HEAD,%LEG,SCBL,SCBA,SCHL,SCHA,SCLL,SCLA,RV,SR,TDL,TDA,TDS,TK ACC,SGBL,SGBA,SGHL,SGHA,SGLL,SGLA,AD,ADTB,ADHG,ADTM,ADTS,SM,Decision,Rnd,Time,hometown,fighting_style,height_x,weight_x,reach_x,leg_reach,height_y,weight_y,reach_y,stance_y,date_of_birth
0,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Sep 10, 2022",Chris Barnett,UFC 279,L,5/7,22/62,1/1,57,107,51,98,53.27%,1,83%,47%,100%,5,5,8,11,0,0,0,0.000,1,2,0,50%,0,0,10,12,0,0,0,0,0,0,0,1,KO/TKO,2,2:24,"St. Louis, United States",Freestyle,75.00,283.00,78.50,43.50,"6' 3""",230 lbs.,"78""",Orthodox,"Oct 25, 1988"
1,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Apr 30, 2022",Andrei Arlovski,UFC Fight Night,L,4/7,68/155,9/9,132,231,93,188,57.14%,0,70%,46%,90%,3,3,9,13,0,1,0,0.000,2,3,0,67%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,5:00,"St. Louis, United States",Freestyle,75.00,283.00,78.50,43.50,"6' 3""",230 lbs.,"78""",Orthodox,"Oct 25, 1988"
2,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Jan 15, 2022",Chase Sherman,UFC Fight Night,W,2/2,10/33,0/1,20,50,20,49,40.00%,0,100%,39%,0%,0,0,0,0,0,0,0,0.000,1,1,0,100%,0,0,8,13,0,0,0,0,0,0,0,1,Submission,1,2:26,"St. Louis, United States",Freestyle,75.00,283.00,78.50,43.50,"6' 3""",230 lbs.,"78""",Orthodox,"Oct 25, 1988"
3,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Jun 12, 2021",Carlos Felipe,UFC 263,L,18/24,85/227,24/26,130,281,130,281,46.26%,0,77%,38%,92%,2,2,1,2,0,0,0,0.000,0,1,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,5:00,"St. Louis, United States",Freestyle,75.00,283.00,78.50,43.50,"6' 3""",230 lbs.,"78""",Orthodox,"Oct 25, 1988"
4,Jake Collier,USA,Light Heavyweight,"6' 3""",265 lbs,10/25/1988 (34),Orthodox,NaN,"Dec 5, 2020",Gian Villante,UFC Fight Night,W,18/28,87/224,16/18,126,277,123,272,45.49%,0,66%,39%,89%,1,1,1,1,0,0,0,0.000,0,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Unanimous,3,5:00,"St. Louis, United States",Freestyle,75.00,283.00,78.50,43.50,"6' 3""",230 lbs.,"78""",Orthodox,"Oct 25, 1988"


    In the conversions below i use ".astype(float).round().astype(pd.Int64Dtype())" 
    in order to change from float to integers even though there are null values

In [205]:
# There are some '-', '--' values in our dataframe as there were on the sites which where scraped, we'll replace them with nan

df.replace('-', np.nan, inplace=True)
df.replace('--', np.nan, inplace=True)

In [206]:
df = df[~df['division'].isnull()] # Remove the one row with no information

# Change height from feet and inches to meters
df['height'] = (df['height'].str.replace('\'', '').str.split(' ').str[0].astype(float) * 0.3048 + 
                df['height'].str.replace('"', '').str.split(' ').str[1].astype(float) * 0.0254)

df['weight'] = df['weight'].str.split(' ').str[1] # Remove lbs from weight

# Keep only the birth year
df['birth_date'] = df['birth_date'].str.split('/').str[-1].str.split(' ').str[0].astype(float).round().astype(pd.Int64Dtype())

df['reach'] = df['reach'].str[:-1].astype(float) * 0.0254 # Convert reach from inches to meters

df['Date'] = pd.to_datetime(df['Date'], format='%m %d, %Y', infer_datetime_format=True) # Convert to year-month-day

In [207]:
df['TSL-TSA'] = df['TSL-TSA'].str[:-1] # Remove %
df.rename(columns={"TSL-TSA": "%TSL-TSA"}, inplace=True) # Rename the column

df['TK ACC'] = df['TK ACC'].str[:-1] # Remove %
df.rename(columns={"TK ACC": "%TK ACC"}, inplace=True) # Rename the column

df['%BODY'] = df['%BODY'].str[:-1] # Remove %
df['%HEAD'] = df['%HEAD'].str[:-1] # Remove %
df['%LEG'] = df['%LEG'].str[:-1] # Remove %

# Time to seconds
df['Time'] = df['Time'].str.split(':').str[0].astype(float) * 60 + df['Time'].str.split(':').str[1].astype(float) 

df['height_x'] = df['height_x'].astype(float) * 0.0254 # Inches to meters
df['reach_x'] = df['reach_x'].astype(float) * 0.0254 # Inches to meters
df['leg_reach'] = df['leg_reach'].astype(float) * 0.0254 # Inches to meters

# Change height to meters
df['height_y'] = (df['height_y'].str.replace('\'', '').str.split(' ').str[0].astype(float) * 0.3048 + 
                df['height_y'].str.replace('"', '').str.split(' ').str[1].astype(float) * 0.0254)

df['weight_y'] = df['weight_y'].str.split().str[0].astype(float) # Remove lbs.
df['reach_y'] = df['reach_y'].str.replace('"', '').astype(float) * 0.0254 # Remove '"' and convert to meters

 # Convert to year-month-day
df['date_of_birth'] = df['date_of_birth'].str.split().str[-1].astype(float).round().astype(pd.Int64Dtype())

In [208]:
# Fix the division column

weight_classes = [
            "Light Heavyweight", "Featherweight", "Bantamweight", "Flyweight",
            "Welterweight", "Women's Strawweight", "Women's Bantamweight", "Catchweight"
            "Middleweight", "Lightweight", "Heavyweight", "Women's Featherweight", "Women's Flyweight"]

df['division'] = df['division'].where(df['division'].isin(weight_classes), other=np.nan)

### We can now combine the columns which are the same stats but from different sources

In [209]:
# Use combine_first to update null elements with values in the same location

df['height'] = (df['height'].combine_first(df['height_x'])
                            .combine_first(df['height_y'])) # Combine values of height, height_x, height_y

df['weight'] = (df['weight'].combine_first(df['weight_x'])
                            .combine_first(df['weight_y'])) # Combine values of weight, weight_x, weight_y

df['birth_date'] = df['birth_date'].combine_first(df['date_of_birth']) # Combine birth dates

df['stance_x'] = df['stance_x'].combine_first(df['stance_y']) # Combine stances
df.rename(columns={'stance_x': 'stance'}, inplace=True) # Rename the column

df['reach'] = (df['reach'].combine_first(df['reach_x'])
                          .combine_first(df['reach_y'])) # Combine reaches



# Drop the collumns which we dont need after combining them

df.drop(['height_x', 'height_y', 'weight_x', 'weight_y', 
         'date_of_birth', 'stance_y', 'reach_x', 'reach_y'], axis=1, inplace=True)

### Lets create some new columns and change the formating a bit more

In [210]:
# Look at the documentation for explanation

df['SDBL'] = df['SDBL/A'].str.split('/').str[0].astype(float).round().astype(pd.Int64Dtype())
df['SDBA'] = df['SDBL/A'].str.split('/').str[1].astype(float).round().astype(pd.Int64Dtype())
df['SDBL/A'] = df['SDBL'] / df['SDBA']

df['SDHL'] = df['SDHL/A'].str.split('/').str[0].astype(float).round().astype(pd.Int64Dtype())
df['SDHA'] = df['SDHL/A'].str.split('/').str[1].astype(float).round().astype(pd.Int64Dtype())
df['SDHL/A'] = df['SDHL'] / df['SDHA']

df['SDLL'] = df['SDLL/A'].str.split('/').str[0].astype(float).round().astype(pd.Int64Dtype())
df['SDLA'] = df['SDLL/A'].str.split('/').str[1].astype(float).round().astype(pd.Int64Dtype())
df['SDLL/A'] = df['SDLL'] / df['SDLA'] 

In [211]:
list_floats = ['weight', '%TSL-TSA', '%BODY' , '%HEAD', '%LEG', '%TK ACC']

list_integers = ['TSL', 'TSA', 'SSL', 'SSA', 'KD', 'SCBL', 'SCBA', 'SCHL', 'SCHA', 'SCLL',
                 'SCLA', 'RV', 'SR', 'TDL', 'TDA', 'TDS', 'SGBL', 'SGBA', 'SGHL', 'SGHA',
                 'SGLL', 'SGLA', 'AD', 'ADTB', 'ADHG', 'ADTM', 'ADTS', 'SM', 'Rnd']

In [212]:
# Convert floats
df[list_floats] = df[list_floats].astype(float)

# Convert integers
df[list_integers] = df[list_integers].astype(float).round().astype(pd.Int64Dtype())

## Pickle the dataframe

In [213]:
with open("D:/data_projects/mma/clean_df_1.pickle", "wb") as file:
    pickle.dump(df, file)

In [6]:
with open("D:/data_projects/mma/clean_df_1.pickle", "rb") as file:
    df = pickle.load(file)

# Database

    In order to create our table on the database made with pgadmin we do some extra steps

In [8]:
# Create an unique ID
df['ID'] = df['Date'].astype(str) + '__' + df['name'] +  '__' + df['Opponent']

# We want the ID to be the first column in our dataframe
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

df = df[cols]

# Replace nans with None
df = df.replace({pd.NaT: None})

# Drop duplicates in ID
df.drop_duplicates(subset='ID', keep="first", inplace=True)

In [100]:
df

,ID,name,country,division,height,weight,birth_date,stance,reach,Date,Opponent,Event,Res.,SDBL/A,SDHL/A,SDLL/A,TSL,TSA,SSL,SSA,%TSL-TSA,KD,%BODY,%HEAD,%LEG,SCBL,SCBA,SCHL,SCHA,SCLL,SCLA,RV,SR,TDL,TDA,TDS,%TK ACC,SGBL,SGBA,SGHL,SGHA,SGLL,SGLA,AD,ADTB,ADHG,ADTM,ADTS,SM,Decision,Rnd,Time,hometown,fighting_style,leg_reach,SDBL,SDBA,SDHL,SDHA,SDLL,SDLA
0,2022-09-10__Jake Collier__Chris Barnett,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988,Orthodox,1.9939,2022-09-10,Chris Barnett,UFC 279,L,0.714286,0.354839,1.0,57,107,51,98,53.27,1,83.0,47.0,100.0,5,5,8,11,0,0,0,0,1,2,0,50.0,0,0,10,12,0,0,0,0,0,0,0,1,KO/TKO,2,144.0,"St. Louis, United States",Freestyle,1.1049,5,7,22,62,1,1
1,2022-04-30__Jake Collier__Andrei Arlovski,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988,Orthodox,1.9939,2022-04-30,Andrei Arlovski,UFC Fight Night,L,0.571429,0.43871,1.0,132,231,93,188,57.14,0,70.0,46.0,90.0,3,3,9,13,0,1,0,0,2,3,0,67.0,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,300.0,"St. Louis, United States",Freestyle,1.1049,4,7,68,155,9,9
2,2022-01-15__Jake Collier__Chase Sherman,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988,Orthodox,1.9939,2022-01-15,Chase Sherman,UFC Fight Night,W,1.0,0.30303,0.0,20,50,20,49,40.00,0,100.0,39.0,0.0,0,0,0,0,0,0,0,0,1,1,0,100.0,0,0,8,13,0,0,0,0,0,0,0,1,Submission,1,146.0,"St. Louis, United States",Freestyle,1.1049,2,2,10,33,0,1
3,2021-06-12__Jake Collier__Carlos Felipe,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988,Orthodox,1.9939,2021-06-12,Carlos Felipe,UFC 263,L,0.75,0.374449,0.923077,130,281,130,281,46.26,0,77.0,38.0,92.0,2,2,1,2,0,0,0,0,0,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Split,3,300.0,"St. Louis, United States",Freestyle,1.1049,18,24,85,227,24,26
4,2020-12-05__Jake Collier__Gian Villante,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988,Orthodox,1.9939,2020-12-05,Gian Villante,UFC Fight Night,W,0.642857,0.388393,0.888889,126,277,123,272,45.49,0,66.0,39.0,89.0,1,1,1,1,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Decision - Unanimous,3,300.0,"St. Louis, United States",Freestyle,1.1049,18,28,87,224,16,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9105,2015-03-14__Beneil Dariush__Daron Cruickshank,Beneil Dariush,USA,Lightweight,1.778,156.0,1989,Southpaw,1.8288,2015-03-14,Daron Cruickshank,UFC 185,W,0.818182,0.25,1.0,30,54,27,51,55.56,0,85.0,37.0,100.0,1,1,1,4,0,0,0,0,1,1,0,100.0,1,1,6,7,0,0,4,2,1,0,1,2,Submission,2,158.0,United States,MMA,1.0287,9,11,6,24,3,3
9106,2014-10-25__Beneil Dariush__Diego Ferreira,Beneil Dariush,USA,Lightweight,1.778,156.0,1989,Southpaw,1.8288,2014-10-25,Diego Ferreira,UFC 179,W,0.75,0.2,0.625,54,97,32,68,55.67,0,60.0,40.0,70.0,2,5,7,11,1,1,0,0,3,6,0,50.0,1,1,8,17,1,1,2,0,2,0,0,0,Decision - Unanimous,3,300.0,United States,MMA,1.0287,3,4,4,20,5,8
9107,2014-08-23__Beneil Dariush__Anthony Rocco Martin,Beneil Dariush,USA,Lightweight,1.778,156.0,1989,Southpaw,1.8288,2014-08-23,Anthony Rocco Martin,UFC Fight Night,W,0.615385,0.166667,0.5,30,65,26,61,46.15,0,71.0,19.0,60.0,9,11,1,2,1,1,0,0,2,3,0,67.0,0,0,0,0,0,0,2,0,1,1,0,1,Submission,2,218.0,United States,MMA,1.0287,8,13,5,30,2,4
9108,2014-04-11__Beneil Dariush__Ramsey Nijem,Beneil Dariush,USA,Lightweight,1.778,156.0,1989,Southpaw,1.8288,2014-04-11,Ramsey Nijem,UFC Fight Night,L,0.5,0.416667,NaN,14,25,9,20,56.00,0,50.0,43.0,0.0,2,4,1,2,0,0,0,0,0,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,KO/TKO,1,260.0,United States,MMA,1.0287,1,2,5,12,0,0


## Function to create the table in our database

In [38]:
def create_table(cur):
    
    create_table_command = (""" 
    CREATE TABLE IF NOT EXISTS mma (
    ID text primary key NOT NULL,
    name text,
    country text,
    division text,
    height float,
    weight float,
    birth_date int,
    stance text,
    reach float,
    Date timestamp,
    Opponent text,
    Event text,
    Res text,
    SDBL_A float,
    SDHL_A float,
    SDLL_A float,
    TSL int,
    TSA int,
    SSL int,
    SSA int,
    TSL_TSA_percent float,
    KD int,
    BODY_percent float,
    HEAD_percent float,
    LEG_percent float,
    SCBL int,
    SCBA int,
    SCHL int,
    SCHA int,
    SCLL int,
    SCLA int,
    RV int,
    SR int,
    TDL int,
    TDA int,
    TDS int,
    TK_ACC_percent float,
    SGBL int,
    SGBA int,
    SGHL int,
    SGHA int,
    SGLL int,
    SGLA int,
    AD int,
    ADTB int,
    ADHG int,
    ADTM int,
    ADTS int,
    SM int,
    Decision text,
    Rnd int,
    Time float,
    hometown text,
    fighting_style text,
    leg_reach float,
    SDBL int,
    SDBA int,
    SDHL int,
    SDHA int,
    SDLL int,
    SDLA int
    );
     """)
    
    cur.execute(create_table_command)

## Create the table

In [39]:
# Connect to the PostgreSQL database
conn = pg2.connect(host="localhost", database='PostgreSQL_mma', user='postgres', password='password')

cur = conn.cursor() # Open a cursor to perform database operations

create_table(cur)

conn.commit() # Commit the changes to the database

cur.close() # Close the cursor object
conn.close() # Close the database connection

## Function to insert data into the table

In [ ]:
def insert_data(cur, data, values):
    
    '''
    Insert data to the table in our database
    
    '''
    
    # Define the SQL statement for inserting data into a table
    insert_query = f"""
    INSERT INTO mma (
        ID,
        name,
        country,
        division,
        height,
        weight,
        birth_date,
        stance,
        reach,
        Date,
        Opponent,
        Event,
        Res,
        SDBL_A,
        SDHL_A,
        SDLL_A,
        TSL,
        TSA,
        SSL,
        SSA,
        TSL_TSA_percent,
        KD,
        BODY_percent,
        HEAD_percent,
        LEG_percent,
        SCBL,
        SCBA,
        SCHL,
        SCHA,
        SCLL,
        SCLA,
        RV,
        SR,
        TDL,
        TDA,
        TDS,
        TK_ACC_percent,
        SGBL,
        SGBA,
        SGHL,
        SGHA,
        SGLL,
        SGLA,
        AD,
        ADTB,
        ADHG,
        ADTM,
        ADTS,
        SM,
        Decision,
        Rnd,
        Time,
        hometown,
        fighting_style,
        leg_reach,
        SDBL,
        SDBA,
        SDHL,
        SDHA,
        SDLL,
        SDLA)
    VALUES ({values});
    """

    # Execute the SQL statement to insert data into the table
    cur.executemany(insert_query, data)

## Insert data to the table

In [ ]:
# Convert the DataFrame to a list of tuples
data = [tuple(x) for x in df.to_numpy()]

# %s ,%s ,%s ,%s , etc.. we use this in our query below
values = ('%s ,'*len(df.columns))[:-2] 


# Connect to the PostgreSQL database
conn = pg2.connect(host="localhost", database='PostgreSQL_mma', user='postgres', password='password')

# Create a cursor object to perform database operations
cur = conn.cursor()

# Insert data to the table
insert_data(cur=cur, data=data, values=values)

# Commit the transaction
conn.commit()

# Close the cursor and database connection
cur.close()
conn.close()

## Test

In [161]:
conn = pg2.connect(host="localhost", database='PostgreSQL_mma', user='postgres', password='password')


df = pd.read_sql_query("""
SELECT *
FROM mma

""", conn)

conn.close()

df

,id,name,country,division,height,weight,birth_date,stance,reach,date,opponent,event,res,sdbl_a,sdhl_a,sdll_a,tsl,tsa,ssl,ssa,tsl_tsa_percent,kd,body_percent,head_percent,leg_percent,scbl,scba,schl,scha,scll,scla,rv,sr,tdl,tda,tds,tk_acc_percent,sgbl,sgba,sghl,sgha,sgll,sgla,ad,adtb,adhg,adtm,adts,sm,decision,rnd,time,hometown,fighting_style,leg_reach,sdbl,sdba,sdhl,sdha,sdll,sdla
0,2022-09-10__Jake Collier__Chris Barnett,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988.0,Orthodox,1.9939,2022-09-10,Chris Barnett,UFC 279,L,0.714286,0.354839,1.000000,57.0,107.0,51.0,98.0,53.27,1.0,83.0,47.0,100.0,5.0,5.0,8.0,11.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,50.0,0.0,0.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,KO/TKO,2.0,144.0,"St. Louis, United States",Freestyle,1.1049,5.0,7.0,22.0,62.0,1.0,1.0
1,2022-04-30__Jake Collier__Andrei Arlovski,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988.0,Orthodox,1.9939,2022-04-30,Andrei Arlovski,UFC Fight Night,L,0.571429,0.438710,1.000000,132.0,231.0,93.0,188.0,57.14,0.0,70.0,46.0,90.0,3.0,3.0,9.0,13.0,0.0,1.0,0.0,0.0,2.0,3.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Decision - Split,3.0,300.0,"St. Louis, United States",Freestyle,1.1049,4.0,7.0,68.0,155.0,9.0,9.0
2,2022-01-15__Jake Collier__Chase Sherman,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988.0,Orthodox,1.9939,2022-01-15,Chase Sherman,UFC Fight Night,W,1.000000,0.303030,0.000000,20.0,50.0,20.0,49.0,40.00,0.0,100.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0,0.0,0.0,8.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Submission,1.0,146.0,"St. Louis, United States",Freestyle,1.1049,2.0,2.0,10.0,33.0,0.0,1.0
3,2021-06-12__Jake Collier__Carlos Felipe,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988.0,Orthodox,1.9939,2021-06-12,Carlos Felipe,UFC 263,L,0.750000,0.374449,0.923077,130.0,281.0,130.0,281.0,46.26,0.0,77.0,38.0,92.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Decision - Split,3.0,300.0,"St. Louis, United States",Freestyle,1.1049,18.0,24.0,85.0,227.0,24.0,26.0
4,2020-12-05__Jake Collier__Gian Villante,Jake Collier,USA,Light Heavyweight,1.905,265.0,1988.0,Orthodox,1.9939,2020-12-05,Gian Villante,UFC Fight Night,W,0.642857,0.388393,0.888889,126.0,277.0,123.0,272.0,45.49,0.0,66.0,39.0,89.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Decision - Unanimous,3.0,300.0,"St. Louis, United States",Freestyle,1.1049,18.0,28.0,87.0,224.0,16.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9025,2015-03-14__Beneil Dariush__Daron Cruickshank,Beneil Dariush,USA,Lightweight,1.778,156.0,1989.0,Southpaw,1.8288,2015-03-14,Daron Cruickshank,UFC 185,W,0.818182,0.250000,1.000000,30.0,54.0,27.0,51.0,55.56,0.0,85.0,37.0,100.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0,1.0,1.0,6.0,7.0,0.0,0.0,4.0,2.0,1.0,0.0,1.0,2.0,Submission,2.0,158.0,United States,MMA,1.0287,9.0,11.0,6.0,24.0,3.0,3.0
9026,2014-10-25__Beneil Dariush__Diego Ferreira,Beneil Dariush,USA,Lightweight,1.778,156.0,1989.0,Southpaw,1.8288,2014-10-25,Diego Ferreira,UFC 179,W,0.750000,0.200000,0.625000,54.0,97.0,32.0,68.0,55.67,0.0,60.0,40.0,70.0,2.0,5.0,7.0,11.0,1.0,1.0,0.0,0.0,3.0,6.0,0.0,50.0,1.0,1.0,8.0,17.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,Decision - Unanimous,3.0,300.0,United States,MMA,1.0287,3.0,4.0,4.0,20.0,5.0,8.0
9027,2014-08-23__Beneil Dariush__Anthony Rocco Martin,Beneil Dariush,USA,Lightweight,1.778,156.0,1989.0,Southpaw,1.8288,2014-08-23,Anthony Rocco Martin,UFC Fight Night,W,0.615385,0.166667,0.500000,30.0,65.0,26.0,61.0,46.15,0.0,71.0,19.0,60.0,9.0,11.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0,3.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,Submission,2.0,218.0,United States,MMA,1.0287,8.0,13.0,5.0,30.0,2.0,4.0
9028,2014-04-11__Beneil Dariush__Ramsey Nijem,B

## Documentation

In [150]:
# 'TSL-TSA' renamd to %TSL-TSA ... percentage, and removed % from values
# 'TK ACC' renamed to %TK ACC ... percentage, and removed % from values
# Added 'SDBL' Significant Distance Body Strikes Landed
#       'SDBA' Significant Distance Body Strike Attempts
#       'SDHL' Significant Distance Head Strikes Landed
#       'SDHA' Significant Distance Head Strike Attempts
#       'SDLL' Significant Distance Leg Strikes Landed
#       'SDLA' Significant Distance Leg Strike Attempts
# % became '_percent' 
# '-' ,'/' became '_'
# Added 'ID' column which is 'date__name__oponent'

# All values converted to floats or integers

distance_header_map = {
                    'SDBL/A': 'Significant Distance Body Strikes Landed-Significant Distance Body Strike Attempts',
                    'SDHL/A': 'Significant Distance Head Strikes Landed-Significant Distance Head Strike Attempts',
                    'SDLL/A': 'Significant Distance Leg Strikes Landed-Significant Distance Leg Strike Attempts',
                    'TSL': 'Total Strikes Landed',
                    'TSA': 'Total Strikes Attempts',
                    'SSL': 'Significant Strikes Landed',
                    'SSA': 'Significant Strikes Attempts',
                    'TSL-TSA': 'Total Strikes Landed-Total Strikes Attempts',
                    'KD': 'Knockdowns',
                    '%BODY': 'Target Breakdown Body',
                    '%HEAD': 'Target Breakdown Head',
                    '%LEG': 'Target Breakdown Leg'}

clinch_header_map = {
                    'SCBL': 'Significant Clinch Body Strikes Landed',
                    'SCBA': 'Significant Clinch Body Strike Attempts',
                    'SCHL': 'Significant Clinch Head Strikes Landed',
                    'SCHA': 'Significant Clinch Head Strike Attempts',
                    'SCLL': 'Significant Clinch Leg Strikes Landed',
                    'SCLA': 'Significant Clinch Leg Strike Attempts',
                    'RV': 'Reversals',
                    'SR': 'Slam Rate',
                    'TDL': 'Takedowns Landed',
                    'TDA': 'Takedowns Attempted',
                    'TDS': 'Takedowns Slams',
                    'TK ACC': 'Takedown Accuracy'}

ground_header_map = {
                    'SGBL': 'Significant Ground Body Strikes Landed',
                    'SGBA': 'Significant Ground Body Strike Attempts',
                    'SGHL': 'Significant Ground Head Strikes Landed',
                    'SGHA': 'Significant Ground Head Strike Attempts',
                    'SGLL': 'Significant Ground Leg Strikes Landed',
                    'SGLA': 'Significant Ground Leg Strikes Attempted',
                    'AD': 'Advances',
                    'ADTB': 'Advance To Back',
                    'ADHG': 'Advance To Half Guard',
                    'ADTM': 'Advance To Mount',
                    'ADTS': 'Advance To Side',
                    'SM': 'Submissions'}